# Recipe Generation using Transformers

Transformers have been shown to outperform custom architectures on many NLP tasks. In text generation particularly, they are able to better capture complex long-range dependencies and generate fluent sentences. However, effective language generation systems must not only create grammatically accurate text but text that is topically consistent and **globally coherent**. 

Recipe generation is an example of a task where such global coherence is needed. Recipes are essentially a sequence of instructions which  have used GRUs and LSTMs.Recipes are a sequence of instructions about how to use a given set of ingredients to create a particular dish. Any recipe generation model needs to keep track of which ingredients from a particular recipe has already been mentioned in the generated text and those which still need to be used. Previous solutions to the global coherence problem have used architectures which combine RNNs like LSTMs and GRUs with a learnt component which tracks the *agenda*, a list of strings which should be mentioned in the output. Inspired by the improved performance of transformers on domain-specific downstream tasks we ask: Can a transformer generate globally coherent text according to an agenda?

To answer this, we focus on the simpler task of recipe completion. Given a recipe title, list of ingredients and a fixed span of text from a recipe, we study the performance of a Neural Checklist model and T5 transformer on generating the remaining parts of the recipe.

## Import Libraries

In [1]:
# Import packages
import os, glob, argparse, logging
import pickle, json
import time, random
import re
from tqdm import tqdm
from itertools import chain
from string import punctuation
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

from helpers.T5 import *
from helpers.checklist import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Installing required packages...
Done!
cuda:0


[nltk_data] Downloading package punkt to /home/advaith/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Some helper functions

In [57]:
# Ensures reproducibility of our results
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

def present_recipes(results):
    gen = list(results['Generated'])
    target = list(results['Target'])
    df = {"Generated": gen, "Target":target}
    df_2 = pd.DataFrame(df)
    return df_2

## Load Dataset

We used the 'Food.com Recipes and Interactions' dataset. The below code loads in the data.

## Baseline - Neural Checklist 

Below we load the model with pretrained weights.

In [ ]:
# Loads Embeddings matrix
emb_mat = np.load('emb_mat.npy')

a_file = open("word2idx.json", "r")
word2idx = json.load(a_file)
a_file.close()

idx2word = {v:k for k,v in word2idx.items()}

# Load Neural Checklist model architecture with weights
model = Model(emb_mat).to(device)
model.load_state_dict(torch.load('model_weights/checklist_weights'))

The below function generates a completed recipe given the goal `g` and the agenda `E`.

## T5 Transformer

The class below contains the essential functions to preprocess and prepare the recipe dataset for fine-tuning the T5 model.

In [5]:
class RecipeDataset(Dataset):
    
    def __init__(self, tokenizer, data_dir, type_path,  max_len=512, mask_percent=0.5, mode='fill'):
        self.data_dir = data_dir
        self.type_path = type_path
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self.ingredients = []
        self.names = []
        self.mode = mode
        self.mask_percent = mask_percent
        
        self.proc = RecipeProcessor()
        self._build()
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, 
                "target_ids": target_ids, "target_mask": target_mask}
  
    def __len__(self):
        return len(self.inputs)
  
    def _build(self):
        if self.type_path == 'train':
            examples = self.proc.get_train_examples(self.data_dir)
        elif self.type_path == 'val':
            examples = self.proc.get_dev_examples(self.data_dir)
        elif self.type_path == 'test':
            examples = self.proc.get_test_examples(self.data_dir)

        for i, example in enumerate(examples):
            if i % 10000 == 0:
                print(i)
            self._create_features(example)
        
    def _create_completion(self, steps):
        
        words = [word for step in steps for word in step.split(" ")]
        total_words = len(words)
        
        if total_words == 0:
          input_words = []
          label = []
        else:
          mask_words = np.round(self.mask_percent*total_words).astype(int)
          input_words = words[:mask_words] + ['<extra_id_0>']
          label = ['<extra_id_0>'] + words[mask_words:] + ['<extra_id_1>']

        label = " ".join(label)
        input_words = " ".join(input_words)
        
        return(input_words, label)
    
    def _create_features(self, example):
    
        ingredients_ = ",".join(example.ingredients)
        name_ = example.name
        
        if self.mode == 'fill':
          masked_, label_ = self._create_masks(example.steps)
        
        elif self.mode == 'complete':
          masked_, label_ = self._create_completion(example.steps)

        elif self.mode == 'shuffle_complete':
          masked_, label_ = self._create_completion(example.steps)  
                                        
        input_ = "name: %s  ingredients: %s masked: %s </s>" % (name_, ingredients_, masked_)
        target =  label_ + " </s>"
        
        # tokenize inputs
        tokenized_inputs = self.tokenizer.batch_encode_plus(
            [input_], max_length = 128, pad_to_max_length=True, truncation=True, return_tensors="pt"
        )
        # tokenize targets
        tokenized_targets = self.tokenizer.batch_encode_plus(
            [target], max_length = 75, pad_to_max_length=True, truncation=True, return_tensors="pt"
        )

        self.inputs.append(tokenized_inputs)
        self.targets.append(tokenized_targets)
        self.ingredients.append(example.ingredients)
        self.names.append(example.name)

### Recipe generator function

The following function `generate_recipes` sets up our fine-tuned T5 model and uses it to generate predictions on sample data.

In [45]:
def generate_recipes(data_dir=None, checkpt_fpath=None, mode = 'fill'):
    
    # Load T5 fine-tuning model with fine-tuned weights
    model = T5FineTuner.load_from_checkpoint(checkpt_fpath)

    # Load T5 tokenizer
    tokenizer = T5Tokenizer.from_pretrained("t5-base")

    # Load Recipe dataset
    dataset =  RecipeDataset(tokenizer, data_dir=data_dir, type_path='test', mode=mode)
    loader = DataLoader(dataset, batch_size=len(dataset), num_workers=2)

    model.model.eval()
    outputs = []
    targets = []

    for i, batch in enumerate(tqdm(loader)):

        # Need to change max. length argument
        outs = model.model.generate(input_ids=batch['source_ids'], 
                                    attention_mask=batch['source_mask'], 
                                    max_length=75)
        dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
        target = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]

        # Store the generated sentences
        outputs.extend(dec)
        targets.extend(target)
    
    # Store results
    results = {'Recipe Title':dataset.names, 'Ingredients List':dataset.ingredients, 'Generated':outputs, 'Target':targets}
  
    return results

### Generate Results

In [46]:
results = generate_recipes(data_dir="data/", checkpt_fpath="exp_11_rc_ef_low_reg.ckpt", mode="shuffle_complete")

/home/advaith/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2073: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/advaith/anaconda3/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:174: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
  0%|          | 0/1 [00:00<?, ?it/s]

0


100%|██████████| 1/1 [00:03<00:00,  3.06s/it]


In [59]:
# print generated sentences
results_df = pd.DataFrame(results)
results_df[['Recipe Title', 'Ingredients List']].head(10)

Recipe Title  \
0  creamy bleu cheese sauce         
1  amazing chocolate cheesecake     
2  grilled herb shrimp              
3  cream cheese icing               
4  protein porridge                 
5  bread machine sage wheat bread   
6  blt ranch dog                    
7  simple vegetarian stew           
8  more than s mores cookies        
9  minty strawberry lemonade        

                                                                                                                                                          Ingredients List  
0  [garlic cloves, onion, cucumber, blue cheese, cream cheese, sour cream, black pepper]                                                                                    
1  [nonfat cottage cheese, liquid egg substitute, sugar, fat free cream cheese, hersheys cocoa powder, vanilla extract]                                                     
2  [large shrimp, garlic cloves, yellow onion, fresh parsley, fresh basil, dry mustard, dijon mustard, kosher salt, fresh ground black pepper, olive oil, lemon, juice of]  
3  [cream cheese, margarine, vanilla, icing sugar]                                                                                                                          
4  [quick-cooking oats, psyllium, protein powder, cinnamon, nutmeg, full cream milk powder, vanilla essence, water]                                                         
5  [milk, shortening, whole wheat flour, bread flour, cornmeal, brown sugar, dried sage, salt, active dry yeast]                                                            
6  [hot dogs, hot dog buns, iceberg lettuce, tomatoes, red onion, buttermilk ranch dressing, bacon]                                                                         
7  [onion, garlic cloves, fresh carrots, white beans, vegetable broth, italian stewed tomatoes]                                                                             
8  [milk chocolate, miniature marshmallows, rice krispies, graham wafers, peanuts, raisins, chocolate]                                                                      
9  [lemon juice, sugar, water, strawberries, mint leaves]

In [58]:
present_recipes(results_df)

Generated  \
0  at least 1 hour before serving serve with crackers                                                                                                                                                                                                                                                                          
1  extract in a bowl and mix well pour into prepared pan and bake for 30 minutes or until a toothpick inserted in center comes out clean cool in pan on a wire rack                                                                                                                                                            
2  baking sheet grill for 5 minutes on each side or until shrimp are pink and cooked through serve with lemon wedges and a green salad                                                                                                                                                                                         
3  ingredients                                                                                                                                                                                                                                                                                                                 
4  minutes, stirring occasionally remove from heat and stir in vanilla serve hot                                                                                                                                                                                                                                               
5  directions and mix well bake at 350 for 1 hour                                                                                                                                                                                                                                                                              
6  the tomatoes, onion, ranch dressing and bacon, and mix well place the hot dogs on a cookie sheet and bake at 350 degrees for about 20 minutes                                                                                                                                                                               
7  and simmer for 20 minutes, stirring occasionally add the cooked vegetables and simmer for another 20 minutes, or until the vegetables are tender serve with a green salad                                                                                                                                                   
8  time into the hot milk chocolate and stir until well blended drop by rounded teaspoonfuls onto a greased cookie sheet and flatten slightly with a fork bake at 350 degrees for 10-12 minutes or until the edges are lightly browned cool on a wire rack for 5 minutes and then transfer to a wire rack to cool completely   
9  2 teaspoons of the lemonade over each strawberry and mint leaf, then top with remaining strawberry and mint leaves cover and freeze until firm, about 2 hours serve with ice cream or whipped cream                                                                                                                         

                                                                                                                                                                                                                                                                         Target  
0  1 hour minimum will keep a few days refrigerated                                                                                                                                                                                                                              
1  process until smooth pour into prepared pan bake 35-40 minutes or until edges are set and center is not jiggly cool completely cover, refrigerate until chilled remove side of pan and serve                                               